# IBDB Webscraping
This file webscrapes publically available data from the Broadway League's [IBDB](https://www.ibdb.com/shows) (Internet Broadway Database) for the purposes of analyzing the impact of Tony Awards outcomes on Broadway Productions.

This file borrows code in its first few chunks and takes general inspiration from the following [Colaboratory Jupyter Notebook](https://colab.research.google.com/drive/1IVwOhBMYay14NkO7kGkrPu0Ij9dSDdEP) by Yaakov Bressler.

In [1]:
import io
import requests
from lxml import etree
from bs4 import BeautifulSoup
import urllib.request
import re
import string
#import json
#import datetime
import pandas as pd
import numpy as np
import urllib
import ast
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

## Scraping the names of the shows we are interested in

We need to create a list of all productions that opened on Broadway between 1979 - 2023, so that we can search for them on the IBDB. To that end, this portion of the notebook, scrapes the names of these productions from a page on [broadwayworld.com](https://www.broadwayworld.com/browseshows.cfm?showtype=BR)

### Create a function that grabs links from a page, using a tag to identify value of link

In [2]:
def getLinks_tagged_fast(url, tag):
    """
    This function finds elements (nodes) at a given url that have attribute 'href' and returns a list of all the 
    urls the href attributes refer to. 
    """
    r = requests.get(url)
    html_doc = r.text
    soup = BeautifulSoup(html_doc, 'html.parser')
    links = []
    # set the opening of each link to be...
    tag = tag
    for link in soup.findAll('a', attrs={'href': re.compile(tag)}):
        links.append(link.get('href'))
    return links

https://www.broadwayworld.com/browseshows.cfm?showtype=BR

The above link is our starting point. It will allow us to get the name of every Broadway production that opened between 1979 and now.

In [3]:
def get_show_links_year(year_url):
    """
    This function webscrapes from a page on broadwayworld.com to get the urls to many webpages for different
    Broadway shows. Each page is for a different show that has been on Broadway. This function scrapes the links for
    every show that opened since 1979. It returns a list of the urls of interest.
    """
    url = year_url
    tag_year = 'browseshows.cfm?'
    #calling previous function to get the links I want for the different years
    years = getLinks_tagged_fast(url, tag_year)[1:]
    page_base = 'https://www.broadwayworld.com/'
    years_loop =[]
    for year in years:
        #focusing on 1979 or later
        if year[-4:].isdigit() and int(year[-4:]) >= 1979:
            years_loop.append(page_base+year)
    
    # Now you have all the years
    tag_show = 'https://www.broadwayworld.com/shows/backstage.php?'
    show_links_nested = []
    for year in years_loop:
        show_links_nested.append(getLinks_tagged_fast(year,tag_show))
    show_links = sum(show_links_nested, [])
    
    return show_links
    
#running function to get my list of links to the productions I want
year_url = 'https://www.broadwayworld.com/browseshows.cfm?showtype=BR'
show_links = get_show_links_year(year_url)

In [4]:
len(show_links)

1877

The above number is the number of different productions that have opened between 1979 and now.

From here, we need to iterate through `show_links` to get the name of every show that has been on Broadway since 1979. Once we have these names, we can search for them on the IBDB.

In [5]:
def get_show_name(url):
    """
    This function takes a url for a given show's page on broadwayworld.com as input and uses an xpath statement to 
    search for the name of the show on the website. It returns the name of the show. 
    """
    response = requests.get(url)
    #make sure url actually exists
    assert response.status_code == 200

    show_html = response.text
    htmlparser = etree.HTMLParser()
    tree = etree.parse(io.StringIO(show_html), parser=htmlparser)
    showroot = tree.getroot()
    #this xml path is the same for every url, it will give you the name of the show for that url
    show_name = showroot.xpath("//span[@itemprop = 'name']/text()")[1]
    
    return show_name

In [6]:
#testing the function on a random url (tried this with several urls)
get_show_name('https://www.broadwayworld.com/shows/backstage.php?showid=6366')

'Oliver!'

In [7]:
#using get_show_name function to make a list of every single show I want to find data on
#took about 10 minutes to run this chunk
show_names = []
for link in show_links:
    show_name = get_show_name(link)
    show_names.append(show_name)
    
#should match length of show_links
len(show_names)

1877

Due to the presence of revivals in the list, some show names appear more than once. To reduce redundancy in our searches, we want to only keep unique show names. This will make the search process more efficient.

In [9]:
unique_show_names = []
for show in show_names:
    if show not in unique_show_names:
        unique_show_names.append(show)    

#print(unique_show_names)     
len(unique_show_names)

1590

In [10]:
#database will not accept searches with & or + in them, so I must replace these with 'and'
for i in range(len(unique_show_names)):
    unique_show_names[i] = unique_show_names[i].replace('+', 'and')
    unique_show_names[i] = unique_show_names[i].replace('&', 'and')

In [11]:
#unique_show_names

## Searching for shows of interest on the IBDB

The next step to is search the [IBDB](https://www.ibdb.com/shows) for the relevant productions. It would take us way too long to do these searches manually, so we need to automate the process using the selenium package. The next several chunks perform these searches.

In [14]:
def search_alt_path(path):
    """
    When the initial path for webscraping (in the next chunk) does not produce results, this function uses an
    alternate path to find the urls that we want. There are two potential alternate paths that can yield results.
    This function has no return value; it edits alt_urls_list in place
    """
    alt_web_elts = driver.find_elements('xpath', path)
    assert len(alt_web_elts) > 0
    alt_results = [elt.get_attribute("href") for elt in alt_web_elts]
    alt_urls_list.extend(alt_results)

In the next chunk, there is a list called `alt_urls_list`. We need this list because not all searches yield the same kind of results page. Most of the time, the search will take you directly to the page for the show you are looking for. However, if there are multiple shows with the same name, you are taken to a page that presents you with links to the different shows, and you need to pick the one you want. For example, when you search for "Hamilton," you are given two results: the hit musical that opened in 2015, and an obscure play of the same name from 1917. You will get similar results if the name of the show you are searching for is contained in the names of other shows. For example, if you search for the play "Wit," you get a very large number of results, most of which have the word "with" somewhere in the title. You can also get undesirable results if the website perceives a spelling error in your search. When this happens, you are taken to a page that says "Did you mean:" and then lists several shows, and usually one of them is the one you are looking for. We need to account for these alternate search results, or else we will miss several productions in the scraping process. Hence, we need a list that contains alternate results, so we can sift through them and get what we want.

In [15]:
#this chunk took about 1:30 hours to run

#the xpath statement that will get us the href for most of the productions we am interested in
prod_xpath = '//div[@id = "nyc-productions" and @data-id = "nyc-productions"]/div/div/div/div/a'#/@href'
#alt_path = f"//div[@data-id = 'shows']/div[@class = 'row']/div/a[text() = {show}]"

#this will be a list all of the urls that we need 
urls_list = []

#not all searches yield the same kind of results page 
alt_urls_list = []

#list of show name searches that failed to produce results 
failed_searches = []

driver = webdriver.Chrome()
#test_shows = ['The King and I', 'Dear Evan Hansen', "Rodgers and Hammerstein's Cinderella",'Spring Awakening']
#test_shows = ["Is He Dead?", 'Sweeney Todd','Awake and Sing!', 'Jerry Springer: The Opera']
for show in unique_show_names:
    driver.get('https://www.ibdb.com/shows/')     #website we are searching from (IBDB)
    search_box = driver.find_element('name','ShowProperName')    #locating the searchbar
    search_box.send_keys(show)       #automating the searches (will store results in a list)
    search_box.submit()
    #search_box.send_keys(Keys.ENTER)  #other way to do above line of code
    web_elts = driver.find_elements('xpath', prod_xpath)
    try:
        #if the initial xpath statement gives us no results, we need to try the alternate paths
        assert len(web_elts) > 0
    except:
        #alt_path is the first option for searches that do not immediately give us the desired results
        try:
            alt_path = f'//div[@data-id = "shows"]/div[@class = "row"]/div/a[text() = "{show}"]'
            search_alt_path(alt_path)
        except:
            #second option for searches that do not immediately give desired results
            try:
                alt_path_2 = f'//p/a[text() = "{show}"]'
                search_alt_path(alt_path_2)
            except:
                #if the search still produces no results, we append the show to the failed searches list
                failed_searches.append(show)

    #getting href I need from each selenium WebElement
    results = [elt.get_attribute("href") for elt in web_elts]
    urls_list.extend(results)


In [36]:
len(alt_urls_list)

353

In [37]:
len(failed_searches)

137

In [21]:
len(urls_list)

2105

The length of urls_list is longer than you might expect because the searches scraped several productions that were pre-1979, even though we made a point excluding these shows when building our list of shows to search for. This happened because some of the shows we search for that had productions in 1979 or later also had productions that opened before 1979. For example, "The King and I" was in our search list because there have been 3 productions of it since 1979. However, there were also two productions prior to 1979, both of which were included in url list. This unwanted productions will be filtered out once we start scraping the data.

In [24]:
#this is the number of shows we could not get results for
#it was much higher before I accounted for alternate paths
len(failed_searches)

137

In [25]:
#getting the desired urls from alt_urls_list
for url in alt_urls_list:
    resp = requests.get(url)
    prod_html = resp.text
    soup = BeautifulSoup(prod_html, 'html.parser')
    tree = etree.HTML(str(soup)) 
    prod_urls = tree.xpath(prod_xpath + '/@href')
    prod_urls = ['https://www.ibdb.com' + elt for elt in prod_urls]
    urls_list.extend(prod_urls)

In [26]:
#once again: many of these urls are for pre 1979 productions. They will be filtered out in the next chunk
len(urls_list)

2897

## Scraping weekly data and tony data

This is the fun part! Now that we have the urls for every production we need (and some we don't need), we can scrape the data directly from each page.

In [27]:
#took about 2 hours to run this chunk
#this LoD (poorly structured because of IBDB layout) will have weekly gross/capacity info
prod_data_list = []

#this LoD will have Tony nom/win info, as well as whether a production was a play, musical, or special
tony_data_LoD = []

#this xpath gives us the name of each award the production was nominated for
noms_path = "//div[@class = 'collapsible-body awards-tab']/div[position() = 1]/div/div/div/h4/text()"

#this xpath gives us the number of awards the production won
wins_path = "//div[@class = 'collapsible-body awards-tab']/div[position() = 1]/div[@class='col s1 right-align']/img[@src = '/Images/award.png']"

#this xpath gives us the year that the production was eligible for awards
#this path only works if the production received noms 
#for productions with 0 noms, we will have to do something else 
year_path = "//div[@class = 'collapsible-body awards-tab']/div[position() = 1]/div[@class = 'col s11']/div/div[@class = 'col s12' and position() = 2]/text()"

#this path gives me the opening date for a production
#it is the path that will be used for productions with 0 noms
alt_year_path = "//div[@class = 'col s5 m3 l5 txt-paddings']/div[@class = 'xt-main-title']/text()"

#this xpath will give me the type of show the production was (musical, play, or special)
type_path = "//div[@class='row wrapper hide-on-med-and-up']/div[@class='col s12 txt-paddings tag-block-compact']/i[position() = 1]/text()"

for url in urls_list:
    resp = requests.get(url)
    prod_html = resp.text
    soup = BeautifulSoup(prod_html, 'html.parser')
    #finding node that has javascript text with our data
    script = soup.find_all('script', type='text/javascript')[1]   #node at [1] is one with our data
    #making script into a string so we can easily parse through it
    js: str = script.text
        
    #need a 'try:' because the next few lines of code will not work for productions with no finanical data 
    #(i.e. pre 1979)
    #this is good because we do not care about productions with no financial data
    try:
        #using a regex to search for the dict we are looking for (i.e. the one that has the weekly data)
        raw_json = re.search('var grossdata = {0:\[.*\] };', js, flags=re.MULTILINE).group(0)
        #[16:-1] to exclude unwanted javascript syntax
        data = ast.literal_eval(raw_json[16:-1])
        #adding key:value pair to dict to keep track of which production is which
        data['production'] = url[41:]
        prod_data_list.append(data)
        
        #scraping Tony info now
        #We must do this within the try because this allows us to skip Tony info for productions with no financial data
        # ^ thus making the code more efficient
        #REACH GOAL: split awards into major/minor categories
        # ^ Based on the layout of the website, it would be very tedious/difficult to do this
        tree = etree.HTML(str(soup)) 
        nominations = tree.xpath(noms_path)
        num_noms = len(nominations)
        num_wins = len(tree.xpath(wins_path))
        try:
            year = int(tree.xpath(year_path)[0][26:30])
        except:
            year = tree.xpath(alt_year_path)[0]
        show_type = tree.xpath(type_path)[0]
        
        #making well-formatted LoD with all of the Tony info
        prod_award_dict = {'production': url[41:], 'nominations': num_noms, 'wins': num_wins, 'type': show_type, 'year': year}
        tony_data_LoD.append(prod_award_dict)
    except:
        pass

In [28]:
tony_data_LoD[:10]

[{'production': 'bad-cinderella-535361',
  'nominations': 0,
  'wins': 0,
  'type': 'Musical',
  'year': 'Mar 23, 2023'},
 {'production': 'dancin-4051',
  'nominations': 7,
  'wins': 2,
  'type': 'Musical',
  'year': 1978},
 {'production': 'dancin-535808',
  'nominations': 1,
  'wins': 0,
  'type': 'Musical',
  'year': 2023},
 {'production': 'camelot-13313',
  'nominations': 2,
  'wins': 0,
  'type': 'Musical',
  'year': 1981},
 {'production': 'camelot-4143',
  'nominations': 0,
  'wins': 0,
  'type': 'Musical',
  'year': 'Nov 15, 1981'},
 {'production': 'camelot-4571',
  'nominations': 0,
  'wins': 0,
  'type': 'Musical',
  'year': 'Jun 21, 1993'},
 {'production': 'camelot-534339',
  'nominations': 5,
  'wins': 0,
  'type': 'Musical',
  'year': 2023},
 {'production': 'el-mago-pop-536773',
  'nominations': 0,
  'wins': 0,
  'type': 'Special',
  'year': 'Aug 20, 2023'},
 {'production': 'fat-ham-535958',
  'nominations': 5,
  'wins': 0,
  'type': 'Play',
  'year': 2023},
 {'production': 

In [29]:
prod_data_list[0][1]

[['Feb 19, 2023',
  '$318,478',
  '-2147483648%',
  '2,796',
  '100%',
  'Feb 19',
  318478.0,
  0.0,
  2796.0,
  2796.0,
  '2',
  '0'],
 ['Feb 26, 2023',
  '$684,822',
  '-2147483648%',
  '9,107',
  '93%',
  'Feb 26',
  684822.0,
  0.0,
  9107.0,
  9786.0,
  '7',
  '0'],
 ['Mar 5, 2023',
  '$568,165',
  '-2147483648%',
  '8,695',
  '89%',
  'Mar 5',
  568165.0,
  0.0,
  8695.0,
  9786.0,
  '7',
  '0'],
 ['Mar 12, 2023',
  '$592,938',
  '-2147483648%',
  '8,663',
  '89%',
  'Mar 12',
  592938.0,
  0.0,
  8663.0,
  9786.0,
  '7',
  '0'],
 ['Mar 19, 2023',
  '$642,196',
  '-2147483648%',
  '8,752',
  '89%',
  'Mar 19',
  642196.0,
  0.0,
  8752.0,
  9786.0,
  '7',
  '0'],
 ['Mar 26, 2023',
  '$633,929',
  '-2147483648%',
  '10,185',
  '91%',
  'Mar 26',
  633929.0,
  0.0,
  10185.0,
  11184.0,
  '3',
  '5'],
 ['Apr 2, 2023',
  '$631,890',
  '-2147483648%',
  '8,978',
  '80%',
  'Apr 2',
  631890.0,
  0.0,
  8978.0,
  11184.0,
  '0',
  '8'],
 ['Apr 9, 2023',
  '$639,391',
  '-2147483648%'

`prod_data_list` is poorly structured and needs to be rearranged before we can convert it into a pandas dataframe. It is currently structured as follows:
- each dictionary in the list represents one production
- each key represents one season of data, with the excpetion of the key we added to represent the production
- the value for each season key is an LoL
- each list in the LoL represents one week of data for that production
- in each of these weekly lists, we care about the values at indeces 0 (date), 4 (weekly capacity), and 6 (weekly gross)

We want to create an LoD in which each dictionary represents one row (week) of data. For example, one row of data in the LoD shoul look like this:

`[{'production': 'the-king-and-i-497593', 'date': 'May 29, 2016', 'capacity':'76%', 'gross': 546476.0},...]`

In [30]:
#creating the desired LoD
prod_LoD = []
#prod_data_list[2][0][0]
for prod in prod_data_list:
    for season in prod:
        for week in prod[season]:
            if type(week) == list:
                #print(week)
                relevant_data = {'production': prod['production'], 'date': week[0], 'capacity': int(week[4][:-1]), 'gross': week[6]}
                prod_LoD.append(relevant_data)

#see what first few rows of data look like
prod_LoD[:5]

[{'production': 'bad-cinderella-535361',
  'date': 'May 28, 2023',
  'capacity': 69,
  'gross': 351163.0},
 {'production': 'bad-cinderella-535361',
  'date': 'Jun 4, 2023',
  'capacity': 77,
  'gross': 384017.0},
 {'production': 'bad-cinderella-535361',
  'date': 'Feb 19, 2023',
  'capacity': 100,
  'gross': 318478.0},
 {'production': 'bad-cinderella-535361',
  'date': 'Feb 26, 2023',
  'capacity': 93,
  'gross': 684822.0},
 {'production': 'bad-cinderella-535361',
  'date': 'Mar 5, 2023',
  'capacity': 89,
  'gross': 568165.0}]

## Make dataframes

Now we are finally ready to convert both of our LoDs into pandas dataframes which we will download to the repository's data folder

In [34]:
tony_data = pd.DataFrame(tony_data_LoD)
tony_data = tony_data.set_index('production')
tony_data

,nominations,wins,type,year
production,,,,
bad-cinderella-535361,0,0,Musical,"Mar 23, 2023"
dancin-4051,7,2,Musical,1978
dancin-535808,1,0,Musical,2023
camelot-13313,2,0,Musical,1981
camelot-4143,0,0,Musical,"Nov 15, 1981"
...,...,...,...,...
a-life-522289,3,0,Play,2020
barnum-3949,10,3,Musical,1980
home-3953,2,0,Play,1980


In [35]:
weekly_data = pd.DataFrame(prod_LoD)
weekly_data = weekly_data.set_index(['production','date'])
weekly_data

capacity     gross
production            date                            
bad-cinderella-535361 May 28, 2023        69  351163.0
                      Jun 4, 2023         77  384017.0
                      Feb 19, 2023       100  318478.0
                      Feb 26, 2023        93  684822.0
                      Mar 5, 2023         89  568165.0
...                                      ...       ...
nuts-3948             Jul 20, 1980        63   46596.0
bent-3823             Jun 8, 1980         36   43510.0
                      Jun 15, 1980        51   57117.0
                      Jun 22, 1980        48   52885.0
                      Jun 29, 1980        57   63754.0

[55318 rows x 2 columns]

In [38]:
#save to data folder
tony_data.to_csv('data/tony_data.csv')
weekly_data.to_csv('data/weekly_data.csv')